In [5]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder,StandardScaler, MinMaxScaler, RobustScaler
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestRegressor

In [6]:
df_train = pd.read_csv('/Users/ybeen/yb/github/convince/dynamic-pricing/data_preprocessing/data/train_dynamic.csv')

In [7]:
df_train

,cost,margin_max,margin_min,name,price,category_id,user_id,margin,requrchase,qurchase,tag
0,25300,0.82,0.25,배 신고 상 15,31625.0,7,1,0.490119,0.370182,1.61213,2.274581
1,25300,0.82,0.25,배 신고 상 15,32890.0,7,1,0.490119,0.370182,1.57200,1.409421
2,25300,0.82,0.25,배 신고 상 15,34155.0,7,1,0.490119,0.370182,1.39900,1.509315
3,25300,0.82,0.25,배 신고 상 15,35420.0,7,1,0.490119,0.370182,1.35400,0.484365
4,25300,0.82,0.25,배 신고 상 15,36685.0,7,1,0.490119,0.370182,1.33300,0.815534
...,...,...,...,...,...,...,...,...,...,...,...
1986,900,1.37,0.25,월동배추 하 10,1890.0,11,1,0.888889,0.466341,0.12400,2.906980
1987,900,1.37,0.25,월동배추 하 10,1935.0,11,1,0.888889,0.466341,0.12000,1.116250
1988,900,1.37,0.25,월동배추 하 10,1980.0,11,1,0.888889,0.466341,0.11900,0.278692
1989,900,1.37,0.25,월동배추 하 10,2025.0,11,1,0.888889,0.466341,0.11700,2.571717


In [47]:
df_train['tag'] = df_train['tag']*100

In [48]:
x_train = df_train.drop(['name', 'margin', 'user_id', 'price'], axis=1)
y_train = df_train['price']
transformer = make_column_transformer(
    (StandardScaler(), ['cost']),
    remainder='passthrough')
transformer.fit(x_train)
print(x_train.columns)
x_train = transformer.transform(x_train)

Index(['cost', 'margin_max', 'margin_min', 'category_id', 'requrchase',
       'qurchase', 'tag'],
      dtype='object')


In [49]:
model = RandomForestRegressor()
model.fit(x_train, y_train)

RandomForestRegressor()

In [50]:
print(model.score(x_train, y_train))

0.9995115582186429


In [143]:
cost = 6000
max_m = 0.70
min_m = 0.2
category_id = 2
requrchase = 0.6
purchase = 1.8
tag = 1
x_test = [[cost, max_m, min_m, category_id, requrchase, purchase, tag]]
x_test = pd.DataFrame(x_test, columns=['cost', 'margin_max', 'margin_min', 'category_id', 'requrchase','qurchase', 'tag'])
x_test = transformer.transform(x_test)

y_predict = model.predict(x_test)

In [144]:
print(y_predict[0])
print(((cost + cost*max_m)+(cost + cost*min_m))/2)
print(f'{cost + cost*min_m} ~ {cost + cost*max_m}')

7841.65
8700.0
7200.0 ~ 10200.0


In [60]:
transformer

ColumnTransformer(remainder='passthrough',
                  transformers=[('standardscaler', StandardScaler(), ['cost'])])

In [317]:
import pickle

pickle.dump(transformer, open('./transformer.pkl', 'wb'))

In [322]:
load_transformer = pickle.load(open('./transformer.pkl', 'rb'))


In [323]:
pickle.dump(model, open('new_price.pkl', 'wb'))

In [324]:
load_model = pickle.load(open('./new_price.pkl', 'rb'))

In [325]:
x_test = [[45000, 30000, 0.266, '사과', 25000]]
x_test = pd.DataFrame(x_test, columns=['최고가격', '최저가격', '계절지수', '카테고리', '원가'])
x_test = load_transformer.transform(x_test)
y_predict = load_model.predict(x_test)

In [326]:
y_predict

array([36942.17780831])